In [12]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.express as px
import plotly
import pandas as pd
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
import io
from base64 import b64encode
import json

In [2]:
df =pd.read_csv("./heat_shock_feature.csv")

In [3]:
df["year"]=pd.to_datetime(df["datetime"]).dt.year
df

,datetime,Tmean,Tmax,Tmin,precip,precip_flg,Sunlight,wind,humidity,cloud_cover,...,other,home,work1,work2,school,indoor,outdoor,road,other_place,year
0,2016-05-01,18.8,26.5,11.7,0.0,1,22.89,2.7,46.0,5.5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
1,2016-05-02,20.2,28.4,13.6,0.0,1,25.80,2.7,56.0,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
2,2016-05-03,19.6,23.6,16.5,11.5,0,14.31,5.7,72.0,9.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
3,2016-05-04,21.3,25.9,16.2,25.0,0,28.29,5.1,48.0,3.5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
4,2016-05-05,20.4,24.5,16.7,0.0,1,27.70,5.1,31.0,3.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2020-09-26,23.1,26.4,20.8,0.0,0,16.87,5.4,72.0,6.3,...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020
2932,2020-09-27,22.1,26.1,20.0,0.0,0,18.68,4.9,65.0,2.5,...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2020
2933,2020-09-28,21.5,26.0,16.9,0.0,1,19.82,2.7,61.0,3.8,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020
2934,2020-09-29,21.8,26.4,17.0,0.0,1,20.92,3.0,71.0,2.8,...,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2020


In [25]:
styles={
        'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll',
    }
}

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__,show_undo_redo=True,external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.Div([
        dcc.Dropdown(id="year-button",options=[{"label":y,"value":y} for y in df["year"].unique()], value=2016,style={"width":"50%"}),
        html.Button(id="download-button",n_clicks=0, children="Downloads HTML")
    ]),
    
    dcc.Graph(
        id='example-graph'
    ),
    
    html.Div([
        html.Pre(id="selected-data",style=styles["pre"],children="print")
    ]),
    
    html.Div([
        html.Button(children="Register Row",id="add-row-button",n_clicks=0  ),
        dash_table.DataTable(
            id="adding-table",
            columns= [
                {"name":"index",
                 "id":"index",
                 "deletable":True,
                 "renamable":True
                },
                {"name":"begin",
                 "id":"begin",
                 "deletable":True,
                 "renamable":True
                },          
                {"name":"end",
                 "id":"end",
                 "deletable":True,
                 "renamable":True
                }        
            ],
            data=[],
            editable=True,
            row_deletable=True,
            export_format="csv"
        )
    ]),
])

#テーブルに行を追加する
@app.callback(
    Output("adding-table","data"),
    Input("add-row-button","n_clicks"),
    State("adding-table","data"),
    State("selected-data","children"),
    prevent_initial_call=True
)
def register_row(n_clicks,data,newrow):
    begin,end=newrow.split(",")
    index=n_clicks
    data.append({"index":index, "begin":begin, "end":end})
    return data

#グラフの更新
@app.callback(
    Output("example-graph","figure"),
    Input("year-button","value")
)
def update_figure(selected_year):
    dfyear=df[df["year"]==selected_year]
    fig=px.line(dfyear, x="datetime", y="Tmean", color="TDFK",line_shape="spline",render_mode="svg").update_traces(mode="lines+markers")
    fig.update_layout(
        xaxis=dict(rangeslider={"visible":True},
                 type="date" ),
        clickmode="event+select"
    ),
    fig.update_layout(modebar_add=["boxselect","drawline"])
    return fig


#グラフから選択した期間の起点と終点を取り出す。
@app.callback(
    Output("selected-data","children"),
    Input("example-graph","selectedData"),
    prevent_initial_call=True
)
def display_selected_data(selectedData):
    #選択範囲に何も点が入っていなかったときに空リストを参照することになりエラーになる
    #この時のエラーハンドリング必要
    if len(selectedData["points"]) ==0:
        return "empty"
    x1=selectedData["points"][0]["x"]    
    x2=selectedData["points"][-1]["x"]
    begin_to_end=f"{x1},{x2}"
    return begin_to_end
#    return json.dumps(selectedData,indent=2)

#グラフをhtml形式で保存する
@app.callback(
     Output("example-graph","fugure"),
    Input("download-button","value"),
    Input("example-graph","figure",)
    ,prevent_initial_call=True
)
def download_figure(n_clicks,figure):
    if n_clicks==0:
        return figure
    else:
        #figure.write_html("./test.html")
        fig=plotly.graph_objects.Figure(data=figure["data"],layout=figure["layout"],skip_invalid=True)
        fig.write_html("./test.html")
        return figure

app.run_server(debug=True,mode="inline")

/home/takato/miniconda3/envs/jupyterlab-debugger/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

